# Retrieving  and cleaning data

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np 
import requests
import time
from lxml import html
from pybart.api import BART # for interacting w/ BART API
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from postgres_login import secret # my postgreSQL password
import psycopg2
import csv
import os
import sys
import io
import warnings
warnings.filterwarnings('ignore')

## Set up data base
If we haven't already, set up the postgres database that will house the data

In [2]:
# Define a database name 
dbname = 'bart_db'
username = 'bkhurley' # change this to your username

In [3]:
# db connection
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://bkhurley@localhost/bart_db


In [4]:
# create the database if it doesn't exist
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


## Get the data 
Retrieve the BART data and store it in the 'bart_db' database

#### Define some functions

In [4]:
# load and append the data into a pandas dataframe
def load_bart_orig_dest_data(urls):
    # generate list of dataframes from the provided csv urls
    data_ls = [pd.read_csv(d, header=None) for d in urls]
    # return single concatinated dataframe
    data = pd.concat(data_ls)
    return data    

#### Retrieve BART data

In [5]:
# the data come from CSVs provided on the web yearly by BART
orig_dest_urls = ['http://64.111.127.166/origin-destination/date-hour-soo-dest-2011.csv.gz',
                  'http://64.111.127.166/origin-destination/date-hour-soo-dest-2012.csv.gz',
                  'http://64.111.127.166/origin-destination/date-hour-soo-dest-2013.csv.gz',
                  'http://64.111.127.166/origin-destination/date-hour-soo-dest-2014.csv.gz',
                  'http://64.111.127.166/origin-destination/date-hour-soo-dest-2015.csv.gz',
                  'http://64.111.127.166/origin-destination/date-hour-soo-dest-2016.csv.gz',
                  'http://64.111.127.166/origin-destination/date-hour-soo-dest-2017.csv.gz']

In [6]:
# get the data and put them into a dataframe
bart_dest_org_df = load_bart_orig_dest_data(orig_dest_urls)

In [7]:
# check out the structure of the raw data
bart_dest_org_df.head()

,0,1,2,3,4
0,2011-01-01,0,12TH,12TH,1
1,2011-01-01,0,12TH,16TH,1
2,2011-01-01,0,12TH,24TH,3
3,2011-01-01,0,12TH,ASHB,2
4,2011-01-01,0,12TH,BAYF,5


### Initial cleaning

In [8]:
# rename columns to something informative
bart_dest_org_df.rename(columns={0: 'date', 1: 'hour', 2: 'orig', 3: 'dest', 4: 'trips_per_hr'},
         inplace=True)

In [9]:
bart_dest_org_df.head()

,date,hour,orig,dest,trips_per_hr
0,2011-01-01,0,12TH,12TH,1
1,2011-01-01,0,12TH,16TH,1
2,2011-01-01,0,12TH,24TH,3
3,2011-01-01,0,12TH,ASHB,2
4,2011-01-01,0,12TH,BAYF,5


### Get route and station data 

For now, I'll limit my analysis to trips originating from or terminating at stations on the PITT line. This is the busiest BART line.

Can get ordered list of stations on the PITT line from the BART's [Route Information API](https://api.bart.gov/docs/route/routeinfo.aspx) and detailed station information from Bart's [Station Information API](https://api.bart.gov/docs/stn/stns.aspx).

#### Route

In [5]:
# access BART API w/ public key
bart = BART(json_format=True)

In [6]:
# request ordered station names for the PITT-MLBR route
route_info_root = bart.route.routeinfo(route=1)
stations = route_info_root['routes']['route']['config']['station']
print(stations)

['PITT', 'NCON', 'CONC', 'PHIL', 'WCRK', 'LAFY', 'ORIN', 'ROCK', 'MCAR', '19TH', '12TH', 'WOAK', 'EMBR', 'MONT', 'POWL', 'CIVC', '16TH', '24TH', 'GLEN', 'BALB', 'DALY', 'COLM', 'SSAN', 'SBRN', 'SFIA', 'MLBR']


#### Stations
I mainly need zip codes for the stations so I can map weather data to each station, but for now I'll keep all of the station information that BART gives me 

In [7]:
# get zip code for each station
stn_info_root = bart.stn.stns()
stn_df = json_normalize(stn_info_root['stations']['station'])
stn_df

,abbr,address,city,county,gtfs_latitude,gtfs_longitude,name,state,zipcode
0,12TH,1245 Broadway,Oakland,alameda,37.803768,-122.271450,12th St. Oakland City Center,CA,94612
1,16TH,2000 Mission Street,San Francisco,sanfrancisco,37.765062,-122.419694,16th St. Mission,CA,94110
2,19TH,1900 Broadway,Oakland,alameda,37.808350,-122.268602,19th St. Oakland,CA,94612
3,24TH,2800 Mission Street,San Francisco,sanfrancisco,37.752470,-122.418143,24th St. Mission,CA,94110
4,ASHB,3100 Adeline Street,Berkeley,alameda,37.852803,-122.270062,Ashby,CA,94703
5,BALB,401 Geneva Avenue,San Francisco,sanfrancisco,37.721585,-122.447506,Balboa Park,CA,94112
6,BAYF,15242 Hesperian Blvd.,San Leandro,alameda,37.696924,-122.126514,Bay Fair,CA,94578
7,CAST,3301 Norbridge Dr.,Castro Valley,alameda,37.690746,-122.075602,Castro Valley,CA,94546
8,CIVC,1150 Market Street,San Francisco,sanfrancisco,37.779732,-122.414123,Civic Center/UN Plaza,CA,94102
9,COLS,7200 San Leandro St.,Oakland,alameda,37.753661,-122.196869,Coliseum,CA,94621


In [8]:
# load weather underground station abbreviations and append to df
weather_stns = pd.read_csv('/Users/bkhurley/git/insight/project/beat_the_crowd/data/weather_stations.csv')
# join weather station abbreviations with BART station df
bart_wu_stn_df = pd.merge(stn_df, weather_stns, on='abbr')
# take a peek at the df
bart_wu_stn_df.head()

,abbr,address,city,county,gtfs_latitude,gtfs_longitude,name,state,zipcode,wu_loc_abbr
0,12TH,1245 Broadway,Oakland,alameda,37.803768,-122.271450,12th St. Oakland City Center,CA,94612,KOAK
1,16TH,2000 Mission Street,San Francisco,sanfrancisco,37.765062,-122.419694,16th St. Mission,CA,94110,KSFO
2,19TH,1900 Broadway,Oakland,alameda,37.808350,-122.268602,19th St. Oakland,CA,94612,KOAK
3,24TH,2800 Mission Street,San Francisco,sanfrancisco,37.752470,-122.418143,24th St. Mission,CA,94110,KSFO
4,ASHB,3100 Adeline Street,Berkeley,alameda,37.852803,-122.270062,Ashby,CA,94703,KOAK


### Store station info in a postgres table

In [11]:
# send to CSV first
bart_wu_stn_df.to_csv(
    '/Users/bkhurley/git/insight/project/beat_the_crowd/data/bart_wu_stations.csv',
    header=False, index=True)

In [16]:
# connect to postgres
conn = psycopg2.connect(database='bart_db', user='postgres', password=secret)
# retrieve the cursor
cur = conn.cursor()

# create the table if it doesn't already exist
query = '''
    CREATE TABLE IF NOT EXISTS bart_station_info (abbr serial PRIMARY KEY, address varchar, 
    city varchar, county varchar, gtfs_latitude float, gtfs_longitude float, name varchar, 
    state char(2), zipcode char(5), wu_loc_abbr char(4));
    '''
cur.execute(query)
conn.commit()
cur.close()

In [19]:
## insert data into database from pandas (can do that here since it's a small dataframe)
bart_wu_stn_df.to_sql('bart_station_info', engine, if_exists='replace')

#### Use the ordered station list from earlier to filter for trips that start and end on the PITT line

In [20]:
pitt_df = bart_dest_org_df.loc[bart_dest_org_df['orig'].isin(stations) &
                               bart_dest_org_df['dest'].isin(stations)]
# Some trips originate and terminate at the same station. 
# Going to remove for now until I figure out what's going on.
pitt_df.drop(pitt_df[pitt_df['orig'] == pitt_df['dest']].index, inplace=True)
pitt_df.reset_index(drop=True, inplace=True) # renumber indices after dropping rows

#### Differentiate northbound from southbound

In [21]:
def get_route_dir(df, stations):
    '''
    Use the ordered list of stations on the PITT line to determine direction 
    '''
    for station in stations:
        station_idx = stations.index(station)
        # label southbound trips, unless already at end of line
        if not station == 'MLBR':
            sb_stations = stations[(station_idx+1):]
            df.loc[((df['orig'] == station) & 
                    df['dest'].isin(sb_stations)), 'direction'] = 'southbound'
        # label northbound trips, unless at beginning of line
        if not station == 'PITT':
            nb_stations = stations[:(station_idx)]
            df.loc[((df['orig'] == station) &
                    df['dest'].isin(nb_stations)), 'direction'] = 'northbound'
    return df

In [22]:
# determine & label nbound/sbound direction of trips
pitt_df['direction'] = '' # initialize column
pitt_df = get_route_dir(pitt_df, stations)

Check that the direction labels are correct

In [23]:
pitt_df.loc[pitt_df['direction'] == 'northbound'].head()

,date,hour,orig,dest,trips_per_hr,direction
3,2011-01-01,0,12TH,CONC,2,northbound
6,2011-01-01,0,12TH,MCAR,12,northbound
9,2011-01-01,0,12TH,PHIL,1,northbound
10,2011-01-01,0,12TH,PITT,1,northbound
12,2011-01-01,0,16TH,12TH,5,northbound


In [24]:
pitt_df.loc[pitt_df['direction'] == 'southbound'].head()

,date,hour,orig,dest,trips_per_hr,direction
0,2011-01-01,0,12TH,16TH,1,southbound
1,2011-01-01,0,12TH,24TH,3,southbound
2,2011-01-01,0,12TH,CIVC,3,southbound
4,2011-01-01,0,12TH,DALY,1,southbound
5,2011-01-01,0,12TH,EMBR,4,southbound


### Calculate rider volume 

In [25]:
def calc_rider_sums(df, col, trip_label):
    '''
    Calculate total trips per date-hour-direction-station combo.
    Rename trip count using trip_label (should be 'entries' or 'exits')
    '''
    df = pitt_df.groupby(['date', 'hour', 'direction', col]).sum()
    df.reset_index(inplace=True)
    df.rename(columns={col: 'station', 'trips_per_hr': trip_label}, inplace=True)
    return df

In [26]:
# calculate entry sums per date-hour-direction-station
enter_df = calc_rider_sums(pitt_df, 'orig', 'entries')
# calculate exits sums per date-hour-direction-station
exit_df = calc_rider_sums(pitt_df, 'dest', 'exits')

In [27]:
# merge enter & exit dfs
enter_exit_df = pd.merge(enter_df, exit_df)

In [28]:
enter_exit_df.head(30)

,date,hour,direction,station,entries,exits
0,2011-01-01,0,northbound,12TH,16,102
1,2011-01-01,0,northbound,16TH,168,26
2,2011-01-01,0,northbound,19TH,19,91
3,2011-01-01,0,northbound,24TH,72,32
4,2011-01-01,0,northbound,BALB,72,5
5,2011-01-01,0,northbound,CIVC,94,89
6,2011-01-01,0,northbound,COLM,24,2
7,2011-01-01,0,northbound,CONC,2,68
8,2011-01-01,0,northbound,DALY,55,8
9,2011-01-01,0,northbound,EMBR,202,137


Would be a more accurate index of volume if I could track the number of entries and exits going up or down the line, but won't worry about that for now.

In [ ]:
# # southbound rider volume when not dealing with first or last station
# #rider_volume = sum(<trips_per_hr w/ origin @ this station>) - sum(<trips_per_hr w/ dest @ this station>)
# station = 'NCON'
# rider_volume = (sum(test_data.loc[test_data['orig'] == station, 'trips_per_hr']) -
#                 sum(test_data.loc[test_data['dest'] == station, 'trips_per_hr']))
# print(rider_volume)

#### Generate some temporal features from the date

In [29]:
# transform the dates to pd datetime objects
enter_exit_df['date'] = pd.to_datetime(enter_exit_df['date'])

In [30]:
# generate weekday and month features
enter_exit_df['day'] = enter_exit_df['date'].dt.weekday_name
enter_exit_df['month'] = enter_exit_df['date'].dt.month

In [31]:
enter_exit_df.head(10)

,date,hour,direction,station,entries,exits,day,month
0,2011-01-01,0,northbound,12TH,16,102,Saturday,1
1,2011-01-01,0,northbound,16TH,168,26,Saturday,1
2,2011-01-01,0,northbound,19TH,19,91,Saturday,1
3,2011-01-01,0,northbound,24TH,72,32,Saturday,1
4,2011-01-01,0,northbound,BALB,72,5,Saturday,1
5,2011-01-01,0,northbound,CIVC,94,89,Saturday,1
6,2011-01-01,0,northbound,COLM,24,2,Saturday,1
7,2011-01-01,0,northbound,CONC,2,68,Saturday,1
8,2011-01-01,0,northbound,DALY,55,8,Saturday,1
9,2011-01-01,0,northbound,EMBR,202,137,Saturday,1


### Insert BART trip data into database

First, write the data to CSV file. Will be much quicker to copy CSV to a table than inserting from pandas, etc.

In [ ]:
bart_csv_fname = '/Users/bkhurley/git/insight/project/beat_the_crowd/data/entries_exits_2011-2017.csv'
enter_exit_df.to_csv(bart_csv_fname, index=True, header=False)

In [ ]:
# connect to postgres
conn = psycopg2.connect(database='bart_db', user='postgres', password=secret)
# retrieve the cursor
cur = conn.cursor()

In [ ]:
# create the table if it doesn't already exist
query = 'CREATE TABLE IF NOT EXISTS bart_entry_exit (id bigserial PRIMARY KEY, date date, hour varchar, direction varchar, station varchar, entries integer, exits integer, day varchar, month varchar);'
cur.execute(query)
conn.commit()
cur.close()

In [279]:
# copy data from CSV to table
cur = conn.cursor()
f = open(bart_csv_fname, 'r')
cur.copy_from(f, 'bart_entry_exit', sep=',', columns=('id',) + tuple(enter_exit_df.columns))
f.close()
conn.commit()
cur.close()

## Scrape weather data covering the relevant time

Define some functions for scraping tasks

In [32]:
# function to return desired table value from url
def get_tbl_value(url, feature):
    # get html tree from page content
    page = ''
    # deal with connection refusal due to number of requests
    # by waiting a few secs
    while page == '':
        try:
            page = requests.get(url)
        except:
            print("Connection refused by the server..")
            print("Going to sleep for 5 seconds")
            print("ZZzzzz...")
            time.sleep(5)
            print("Wake up, try again...")
            continue
    tree = html.fromstring(page.content)
    # get column values associated with this feature
    row_data = tree.xpath(
        "//tr[contains(., '%s')]//td//" % feature +
        "span[@class='wx-value']//text()")
    # deal with missing or anomalous data
    if not row_data:
        tbl_val = 'NULL'
    elif row_data[0] in ['', '-']:
        tbl_val = 'NULL'
    elif row_data[0] == 'T': 
        # WU uses T to denote trace of participation. 
        # I will treat that as 0.0 inches
        tbl_val = 0.0
    else:
        # convert str to numerical value
        tbl_val = float(row_data[0])                
    return tbl_val

# function to build timestamp
def make_timestamp(m, d, y):
    # Format month
    if len(str(m)) < 2:
        mStamp = '0' + str(m)
    else:
        mStamp = str(m)
    # Format day
    if len(str(d)) < 2:
        dStamp = '0' + str(d)
    else:
        dStamp = str(d)
    # Build timestamp
    time_stamp = '%s-%s-%s' % (str(y), mStamp, dStamp)
    return time_stamp

### Iteratively scrape weather history pages
Iterate through Weather Underground's weather history page for each day within the date range (2006-2017). I use `lxml` library combined with XPath syntax to parse the Weather Undergound XML tables and find the weather values of interest. 

As each day of weather history is scraped, the data are interatively inserted into the postgreSQL table `weatherhistory_raw`

In [33]:
# set to 1 if wish to overwrite existing data file
overwrite_weather_data = 1
output_fname = '/Users/bkhurley/git/insight/project/beat_the_crowd/data/wu_weather_history.csv'
# only scrape if we don't already have the data on file

weath_stn_names = bart_wu_stn_df['wu_loc_abbr'].unique()

if not (os.path.isfile(output_fname) | overwrite_weather_data==0):
    
    # initialize output file
    output_file = open(output_fname, "w")
    output_writer = csv.writer(output_file)
    
    # let's time this process
    start = time.time()

    # define the critical rows that we want from the table weather history page
    out_cols = ['timestamp','wu_loc_abbr','Mean Temperature','Max Temperature','Min Temperature',
                'Precipitation']
    ## define the critical rows that we want from the weather history table
    wu_rows = out_cols
    # write feature names as first row in output file
    output_writer.writerow(out_cols)

    # scrape weather history for each weather station
    for stn in weath_stn_names:    
        # Iterate through year, month, and day starting 2011 ending 2017
        for y in range(2011, 2018):
            for m in range(1, 13):
                for d in range(1, 32):
                    # new row array for each day
                    row_array = []

                    # Check if leap year
                    if y%400 == 0:
                        leap = True
                    elif y%100 == 0:
                        leap = False
                    elif y%4 == 0:
                        leap = True
                    else:
                        leap = False
                    # Check if already gone through month
                    if (m == 2 and leap and d > 29):
                        continue
                    elif (m == 2 and d > 28):
                        continue
                    elif (m in [4, 6, 9, 11] and d > 30):
                        continue

                    # Build timestamp
                    timestamp = make_timestamp(m=m, d=d, y=y)
                    row_array.append(timestamp)
                    # now append weather station name
                    row_array.append(stn)
                    # Generate weather history url for this date
                    url = ("https://www.wunderground.com/history/airport/%s/" % stn +
                           str(y)+ "/" + str(m) + "/" + str(d) + "/DailyHistory.html")
                    # Iterate through each desired weather feature and 
                    # find corresponding value from weather history XML 
                    # table
                    for ifeat in range(1, (len(wu_rows))):
                        row_array.append(get_tbl_value(url=url, feature=wu_rows[ifeat]))
                    # write a row of data for each effort
                    output_writer.writerow(row_array)

                # update status as we work through each month/year
                sys.stdout.write(
                    'Scraped weather records for Month: %d, Year: %d, Weather Station: %s\n' % 
                    (m, y, stn))

    # Done getting data!
    output_file.close()
    sys.stdout.write('\n\nFinished scraping all weather data! Data written to file: %s\n' % 
                     output_fname)
# end the timer
end = time.time()
elapsed = end-start
print(elapsed)

Scraped weather records for Month: 1, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 2, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 3, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 4, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 5, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 6, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 7, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 8, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 9, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 10, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 11, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 12, Year: 2011, Weather Station: KOAK
Scraped weather records for Month: 1, Year: 2012, Weather Station: KOAK
Scraped weather records for Month: 2, Year: 2012, Weather Sta

Scraped weather records for Month: 7, Year: 2013, Weather Station: KSFO
Scraped weather records for Month: 8, Year: 2013, Weather Station: KSFO
Scraped weather records for Month: 9, Year: 2013, Weather Station: KSFO
Scraped weather records for Month: 10, Year: 2013, Weather Station: KSFO
Scraped weather records for Month: 11, Year: 2013, Weather Station: KSFO
Scraped weather records for Month: 12, Year: 2013, Weather Station: KSFO
Scraped weather records for Month: 1, Year: 2014, Weather Station: KSFO
Scraped weather records for Month: 2, Year: 2014, Weather Station: KSFO
Scraped weather records for Month: 3, Year: 2014, Weather Station: KSFO
Scraped weather records for Month: 4, Year: 2014, Weather Station: KSFO
Scraped weather records for Month: 5, Year: 2014, Weather Station: KSFO
Scraped weather records for Month: 6, Year: 2014, Weather Station: KSFO
Scraped weather records for Month: 7, Year: 2014, Weather Station: KSFO
Scraped weather records for Month: 8, Year: 2014, Weather Sta

Scraped weather records for Month: 1, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 2, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 3, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 4, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 5, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 6, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 7, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 8, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 9, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 10, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 11, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 12, Year: 2016, Weather Station: KHWD
Scraped weather records for Month: 1, Year: 2017, Weather Station: KHWD
Scraped weather records for Month: 2, Year: 2017, Weather Sta

Scraped weather records for Month: 7, Year: 2011, Weather Station: KLVK
Scraped weather records for Month: 8, Year: 2011, Weather Station: KLVK
Scraped weather records for Month: 9, Year: 2011, Weather Station: KLVK
Scraped weather records for Month: 10, Year: 2011, Weather Station: KLVK
Scraped weather records for Month: 11, Year: 2011, Weather Station: KLVK
Scraped weather records for Month: 12, Year: 2011, Weather Station: KLVK
Scraped weather records for Month: 1, Year: 2012, Weather Station: KLVK
Scraped weather records for Month: 2, Year: 2012, Weather Station: KLVK
Scraped weather records for Month: 3, Year: 2012, Weather Station: KLVK
Scraped weather records for Month: 4, Year: 2012, Weather Station: KLVK
Scraped weather records for Month: 5, Year: 2012, Weather Station: KLVK
Scraped weather records for Month: 6, Year: 2012, Weather Station: KLVK
Scraped weather records for Month: 7, Year: 2012, Weather Station: KLVK
Scraped weather records for Month: 8, Year: 2012, Weather Sta

Scraped weather records for Month: 1, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 2, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 3, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 4, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 5, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 6, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 7, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 8, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 9, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 10, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 11, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 12, Year: 2014, Weather Station: KPAO
Scraped weather records for Month: 1, Year: 2015, Weather Station: KPAO
Scraped weather records for Month: 2, Year: 2015, Weather Sta

Scraped weather records for Month: 7, Year: 2016, Weather Station: KNUQ
Scraped weather records for Month: 8, Year: 2016, Weather Station: KNUQ
Scraped weather records for Month: 9, Year: 2016, Weather Station: KNUQ
Scraped weather records for Month: 10, Year: 2016, Weather Station: KNUQ
Scraped weather records for Month: 11, Year: 2016, Weather Station: KNUQ
Scraped weather records for Month: 12, Year: 2016, Weather Station: KNUQ
Scraped weather records for Month: 1, Year: 2017, Weather Station: KNUQ
Scraped weather records for Month: 2, Year: 2017, Weather Station: KNUQ
Scraped weather records for Month: 3, Year: 2017, Weather Station: KNUQ
Scraped weather records for Month: 4, Year: 2017, Weather Station: KNUQ
Scraped weather records for Month: 5, Year: 2017, Weather Station: KNUQ
Scraped weather records for Month: 6, Year: 2017, Weather Station: KNUQ
Scraped weather records for Month: 7, Year: 2017, Weather Station: KNUQ
Scraped weather records for Month: 8, Year: 2017, Weather Sta

## Insert weather data into database

Just realized that there's no index column in the csv generated above. Let's overwrite it with a file with an index column. Also, while we have it in a dataframe, I'll generate a binary rain/no-rain feature

In [56]:
weather_csv_fname = '/Users/bkhurley/git/insight/project/beat_the_crowd/data/wu_weather_history.csv'
weather_df = pd.read_csv(weather_csv_fname)

In [57]:
weather_df.head()

,0,1,2,3,4,5,6
0,1/1/11,KOAK,50.0,53,46,0.08,1.0
1,1/2/11,KOAK,49.0,52,45,0.37,1.0
2,1/3/11,KOAK,49.0,57,41,0.00,0.0
3,1/4/11,KOAK,45.0,55,35,0.00,0.0
4,1/5/11,KOAK,47.0,57,37,0.00,0.0


In [58]:
# generate a binary rain feature
weather_df['rain'] = np.nan
weather_df.loc[weather_df['Precipitation']>0., 'rain'] = 1
weather_df.loc[weather_df['Precipitation']==0., 'rain'] = 0
weather_df['rain'] = pd.to_numeric(weather_df.iloc[:, 6], downcast='integer')

In [59]:
# write it back out to csv. Will use this one to copy into Postgres table
weather_df.to_csv(weather_csv_fname, index=True, header=False)

In [104]:
# connect to postgres 
conn = psycopg2.connect(database='bart_db', user='postgres', password=secret)
cur = conn.cursor()

In [105]:
# create the table if it doesn't already exist
query = '''
    CREATE TABLE IF NOT EXISTS weather (id bigserial PRIMARY KEY, datestamp date, 
    wu_loc_abbr char(4), mean_temperature float, max_temperature float, 
    min_temperature float, precipitation float, rain integer);
'''
cur.execute(query)
conn.commit()
cur.close()

In [106]:
# copy data from CSV to table
cur = conn.cursor()
f = open(weather_csv_fname, 'r')
cur.copy_from(f, 'weather', sep=',', columns= ('id', 'datestamp', 'wu_loc_abbr', 'mean_temperature', 'max_temperature', 
                                      'min_temperature', 'precipitation', 'rain'))
f.close()
conn.commit()
cur.close()

In [108]:
weather_df.head()

,0,1,2,3,4,5,6,rain
0,1/1/11,KOAK,50.0,53,46,0.08,1.0,1.0
1,1/2/11,KOAK,49.0,52,45,0.37,1.0,1.0
2,1/3/11,KOAK,49.0,57,41,0.00,0.0,0.0
3,1/4/11,KOAK,45.0,55,35,0.00,0.0,0.0
4,1/5/11,KOAK,47.0,57,37,0.00,0.0,0.0


In [110]:
# commit database changes and close connection
conn.close()